References: 
<br>https://365datascience.com/tutorials/python-tutorials/market-basket-analysis/ 
<br>https://towardsdatascience.com/data-mining-market-basket-analysis-with-apriori-algorithm-970ff256a92c/

In [109]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd

In [110]:
df_orders = pd.read_csv('order_details.csv')

In [111]:
df_pizzas = pd.read_csv('pizzas.csv')

In [112]:
df_pizza_types = pd.read_csv('pizza_types.csv', encoding='latin1')

In [113]:
df_merged_orders_and_pizzas = pd.merge(df_pizzas, df_orders, on='pizza_id', how='inner')

In [114]:
df_mba = pd.merge(df_merged_orders_and_pizzas, df_pizza_types, on='pizza_type_id', how='inner') 
df_mba = df_mba[['order_id', 'name']]

In [115]:
# Remove order_id with single item
df_mba = df_mba[df_mba.groupby('order_id')['order_id'].transform('count') > 1]                                                                                   

In [116]:
# Create a matrix table such that order_id represents the row while pizza name represents the column
transactions_matrix = pd.crosstab(df_mba['order_id'], df_mba['name'])

In [117]:
# To convert qty bought into 0 or 1 instead of the exact qty --> 0 = item was not purchased, 1 = item was purchased 
def encode(item_freq):
    bought = 0
    if item_freq > 0:
        bought = 1
    return bought

# Apply the the function element-wise
basket_input = transactions_matrix.map(encode)

# Convert values to boolean 
basket_input = basket_input.astype(bool)

In [ ]:
# Apply apriori algorithm
frequent_itemsets = apriori(basket_input, min_support=0.001, use_colnames=True)
# Apply association mining rules to find association between pizzas
result = association_rules(frequent_itemsets, metric="lift")


In [ ]:
# Sort product pairs by highest support, confidence and lift values respectively
result = result.sort_values(["support", "confidence", "lift"],axis = 0, ascending = False)
result

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
620,(The Hawaiian Pizza),(The Thai Chicken Pizza),0.140239,0.140771,0.020534,0.146421,1.040134,1.0,0.000792,1.006619,0.044879,0.078832,0.006575,0.146144
621,(The Thai Chicken Pizza),(The Hawaiian Pizza),0.140771,0.140239,0.020534,0.145867,1.040134,1.0,0.000792,1.006589,0.044907,0.078832,0.006546,0.146144
38,(The Barbecue Chicken Pizza),(The Pepperoni Pizza),0.142596,0.145867,0.020458,0.143467,0.983547,1.0,-0.000342,0.997198,-0.019137,0.076334,-0.002810,0.141858
39,(The Pepperoni Pizza),(The Barbecue Chicken Pizza),0.145867,0.142596,0.020458,0.140250,0.983547,1.0,-0.000342,0.997271,-0.019209,0.076334,-0.002736,0.141858
602,(The Hawaiian Pizza),(The Pepperoni Pizza),0.140239,0.145867,0.019697,0.140456,0.962904,1.0,-0.000759,0.993705,-0.042887,0.073937,-0.006335,0.137746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,(The Mediterranean Pizza),(The Brie Carre Pizza),0.057799,0.030421,0.001901,0.032895,1.081332,1.0,0.000143,1.002558,0.079829,0.022026,0.002552,0.047697
140,(The Brie Carre Pizza),(The Italian Vegetables Pizza),0.030421,0.060461,0.001825,0.060000,0.992377,1.0,-0.000014,0.999510,-0.007860,0.020495,-0.000491,0.045094
141,(The Italian Vegetables Pizza),(The Brie Carre Pizza),0.060461,0.030421,0.001825,0.030189,0.992377,1.0,-0.000014,0.999761,-0.008109,0.020495,-0.000239,0.045094
132,(The Brie Carre Pizza),(The Green Garden Pizza),0.030421,0.061678,0.001597,0.052500,0.851199,1.0,-0.000279,0.990314,-0.152756,0.017647,-0.009781,0.039197
